
# Final Project for ANN - Osnabrück 2019
## Maren, Sophia & Malin - Group 4

 
 
 

### Dies ist ein RNN, welches ein "word-level-model" ist. Als Input nimmt es eine .txt Datei mit Tweets einer Partei. Als Output generiert es einen neuen Tweet dieser Partei von einer beliebig wählbaren Länge.


In [93]:
# Pakete die wir brauchen
import numpy as np
import tensorflow as tf
import random 
import matplotlib.pyplot as plt
import scipy.io
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import pandas as pd
import re
import string

# for 4th way to do it
import nltk
#from nltk.tokenize import word_tokenize
from nltk import word_tokenize
from nltk.tokenize import TreebankWordTokenizer

In [94]:
# Funktionen, die wir verwenden

# im "additonal cleaning" bearbeiten wir die .txt files nach. Dies sind Dinge, die uns erst nach dem Erstellen der 
# .txt Dateien aufgefallen sind. Das Erstellen der Files hat so lange gedauert, dass es uns leichter erschien, dies 
# nachträglich zu machen, als das Preprocessing nochmal mit der erweiterten Funktion laufen zu lassen,
# Dennoch ist uns bewusst, dass das "sauberer" w#re
# wir löschen alle html links, dinge die von @ gefolgt sind, einige sonderzeichen und ungewünschte Satzzeichen
def additional_cleaning(text):
    # löscht unerwünschte Zeichen
    text = re.sub(r"…", "", text)
    text = re.sub(r". . . ", "", text)
    text = re.sub(r"r'\S+", "", text)
    #text = re.sub(r"#", "# ", text)
    text = re.sub(r"&amp", "", text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"”", "", text)
    text = re.sub(r"’", "", text)
    text = re.sub(r"•", "", text)
    #text = re.sub(r"+", "", text)
    text = re.sub(r"»", "", text)
    text = re.sub(r"|", "", text)
    text = re.sub(r"«", "", text)
    text = re.sub(r"%", "", text)
    text = ''.join( c for c in text if  c not in '],[,/,:,&,_,1,2,3,4,5,6,7,8,9,0,\,&,;,-,),(,?,!')
   
    # wir möchten alle Buchstaben klein machen, um das Vokabular zu verringern
    text = text.lower()
    

    return text

### Einlesen der Daten

In [95]:
# Öffnen unserer .txt files
text_char = open("GRUENE.txt",'r').read()

# andere Wege um die Datei zu öffnen, abhängig davon, ob alle benötigten Pakete installiert sind

#2rst way
#with open("tweets.txt", 'rb') as f:
#text_char = f.read()
    
#3rd way - klappt am häufigsten, wirft aber smileys etc weg
#with open("GRUENE.txt", encoding="utf8", errors='ignore') as f:
#    text_char = f.read()

In [96]:
# wendet die oben definierte additional_cleaning Funktion auf den Text an
input_text = additional_cleaning(text_char)

In [97]:
# Dies printet den Inhalt der .txt Datei, um einen Eindruck davon zu bekommen,
#wie der Input aussieht
print( "Input text: {}".format (input_text))

Input text:  „verlierer sind verbraucher und umwelein kommentar zum #dieselgipfel von ist mehr als nur glasfaser verbuddeln rahmensetzung  gezielte foerderung guter ideen ausgespaeht geschwaerzt vertuscht diskussion zu geheimdiensten mit und es ist zeit voran zu gehen . anpacken statt aussitzen . fuer eine mutige gruene politik . auf in den wahlkampf #darumgrün  wir diskutieren das wahlprogramm . sieht die praeambel als gute grundlage in die offensive zu gehen .  so geht echte radlhauptstadt . take that gut umweltaktion des kidsclub am stadion . genug schwarzgeaergert . zukunft ist waehlbar #darumgrün  in berlinmitte erststimme mutlu  zweitstimme nun zum infostand zur #btw von in die innenstadt #leipzig . #zukunftwirdausmutgemacht die schirmherrinnen des lernlabors #morgenmehr trude #simonsohn und pressmitteilung starbucks in #berlinmitte getestet  die sprechen deutsch  uff weshalb sie die vorschlaege z antiterrorpolitik ablehnt hat mir gesagt bin mal gespannt wie der vergleich koavert

### Generieren des Vokabulars

In [98]:
# Unterschiedliche Ansätze

# 1.st way to do it - hier bleiben keine Punkte drin
#tokenized_text = list(input_text.split(" "))
#vocab = set(tokenized_text)

# 2nd way to do it - einzelne Buchstaben und Satzzeichen sind weg
#tokenizer = RegexpTokenizer(r'\w+')
#tokenized_text = list(tokenizer.tokenize(text_char))
#vocab = set(tokenized_text)

# 3rd way to do it - this keeps all empty spaces, dots ... as "words" - sehr langsam, lernt Leerzeichen
#p2 = re.compile(r'(\W+)')
#tokenized_text = p2.split(input_text)

# 4th way to do it - keeps smileys, Leerzeichen werden manuell am ende wieder eingefügt
vocab = TreebankWordTokenizer()
tokenized_text = vocab.tokenize(input_text)

vocab = set(tokenized_text)
vocab_size = len(vocab)

print("vocabulary size: {}".format(vocab_size))

text_length = len(tokenized_text)
print("text length: {}".format(text_length))

vocabulary size: 3684
text length: 10962


### Erstellen von einer Übersetzung Wort - ID

In [99]:
# Erstelle dictionaries um zwischen Wörtern und ihren IDs (Nummern) hin und her zu wechseln
word_to_id = { word:i for i, word in enumerate(vocab)}
id_to_word = {i:word for i, word in enumerate(vocab)}


# Übersetze die gesammte .txt Datei aller Tweets in Nummern (= die zugehörigen IDs der Wörter)
text_as_id = [word_to_id[c] for c in tokenized_text]

print("Translation word:id : {}".format(word_to_id))

Translation word:id : {'so': 0, 'lauter': 1, 'polizeirevier': 2, 'industrie': 3, 'offensive': 4, 'unterstuetzen': 5, 'geschaft': 6, 'zuerst': 7, 'regiert': 8, 'kamerarundgang': 9, 'offenbar': 10, 'aufklaerung': 11, 'jamaika': 12, 'aussitzen': 13, 'sondersitzung': 14, 'scandale': 15, 'wahlwatch': 16, 'mobilitaet': 17, 'einwanderungsgesetz': 18, 'bielefeldjahnplatz': 19, 'ne': 20, 'umsetzbare': 21, 'leere': 22, 'voellig': 23, 'bildung': 24, 'metern': 25, 'natuerlich': 26, 'unkrautvernichter': 27, 'beschreibt': 28, 'stuttgart': 29, 'terroristen': 30, 'wajahre': 31, 'hindern': 32, 'selbst': 33, '🌩️': 34, 'bring': 35, 'morde': 36, 'train': 37, 'ausschuss': 38, 'naechsten': 39, 'anordnen': 40, 'outline': 41, 'denn': 42, 'wuenscht': 43, 'termin': 44, 'political': 45, 'antworten': 46, 'feministischen': 47, 'albtour': 48, 'kefayawar': 49, 'diskussionsrunde': 50, 'alt': 51, 'gerechtfertigt': 52, 'voegel': 53, 'alle': 54, 'verstanden': 55, 'taeter': 56, 'chef': 57, 'staat': 58, 'jahrelang': 59, '

In [100]:
# Hier legen wir fest, wie lang die Sequenz ist, die wir beim Lernen auf einmal einlesen
# Da das .txt file, welches wir als Input benutzen aus vielen einzelnen Tweets besteht
# Und diese nicht zwangsläufig zusammenhängen haben wir die Länge einer Input Sequenz
# relativ kurz gewählt
seq_length = 30

# Generate the dataset.
input_data = []
target_data = []
for i in range(text_length-seq_length):
    input_data.append(text_as_id[i:i+seq_length])
    target_data.append(text_as_id[i+1:i+seq_length+1])

#brauchen wir das?    
#input_data = np.cast(input_data, dtype = np.int32)


## Das Model
### Zuerst wird der Input definiert und der Iterator initialisiert

In [101]:
# Zu Beginn eines jeden Models müssen wir den Graph zurücksetzen
tf.reset_default_graph()

# Wir kreiren das Trainingsdatenset. 
# Ein Validation Dataset benötigen wir diesmal nicht, da wir in einem RNN 
# keine Labels haben
dataset = tf.data.Dataset.from_tensor_slices((input_data,target_data))

# Erstellen des Iterators. Da unsere .txt Datei nicht allzu lang ist trainieren wir nicht 
# an einzelnen batches, sondern nehmen den ganzen Text auf einmal als Input
iterator = tf.data.Iterator.from_structure(dataset.output_types,dataset.output_shapes)


In [102]:
# Anhand der iterator.get_next() Methode bekommen wir immer Zugriff auf das nächste 
# Element des Iterators. Der Output dieser methode gibt uns eine Liste, die an der ersten Stelle
# die Input Daten enthält, und an zweiter Stelle die zugehörigen IDs
next_batch = iterator.get_next()

# Hier wird der Iterator initialisiert und die Trainigsdaten werden rein geladen
iterator_init_op = iterator.make_initializer(dataset)

# Die Input daten und die target IDs werden definiert
input_data = next_batch[0]
target_data = next_batch[1]

# Wir machen aus den Input daten one-hot Vektoren
# Dies benötigen wir später, um den Loss zu berechnen. 
# Ein one-hot Vektor ist so lang, wie wir Wörter in 
#unserem vocabulary haben, daher ist die Tiefe auch "vocab_size"
hot_input = tf.one_hot(input_data, depth=vocab_size, on_value=1, off_value=0 )
hot_target = tf.one_hot(target_data, depth=vocab_size,  on_value=1, off_value=0 )

# die one-hot Vektoren müssen nun noch gecastet werden,
# damit eine Berechnung mit den weights möglich ist
hot_input = tf.cast(hot_input, dtype=tf.float32)
hot_target = tf.cast(hot_target, dtype=tf.float32)

print(hot_input)
print(hot_target)

# Nun muss noch ein Placeholder für den hidden state kreirt werden
# als shape benötigen wir [1, die Anzahl an hidden _ neurons] - die Anzahl ist hier frei wählbar
nr_hidden_neurons = 100
hidden_state = tf.placeholder(shape=[1, nr_hidden_neurons], dtype=tf.float32)
print(hidden_state)

Tensor("Cast:0", shape=(30, 3684), dtype=float32)
Tensor("Cast_1:0", shape=(30, 3684), dtype=float32)
Tensor("Placeholder:0", shape=(1, 100), dtype=float32)


## RNN - Hier wird die RNN Zelle definiert

In [103]:
# Im Prizip haben wir nur eine Art von"hidden layer", welche entfaltet wird, wenn wir 
# in das Model rein gehen. Es gibt dann so viele hidden layers wie Worte, die wir als Input
# ins Model reingeben

with tf.variable_scope("simple_RNN_layer", reuse=tf.AUTO_REUSE) as scope:
    
    # Lege den neuen hidden state fest
    new_hidden = hidden_state   
    
    # Erstellen von leeren Listen um die hidden states und die logits abzuspeichern 
    hidden_list = []
    logits = []
    
    # Definiere die "weights" - in einem RNN gibt es input weights, hidden weights und output weights
    weights_in = tf.Variable(tf.random_normal(shape = (vocab_size,nr_hidden_neurons), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    weights_hidden = tf.Variable(tf.random_normal(shape = (nr_hidden_neurons, nr_hidden_neurons), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    weights_out = tf.Variable(tf.random_normal(shape = (nr_hidden_neurons, vocab_size), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    
    # Definiere die "biases" - in einem RNN gibt es biases in der hidden layer und in der output layer
    bias_hidden = tf.Variable(tf.random_normal(shape = (1,nr_hidden_neurons), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    bias_out = tf.Variable(tf.random_normal(shape = (1,vocab_size), mean = 0.0, stddev = 0.1), dtype = tf.float32)
    
   
    # Hier legt sich die Tiefe unseres RNNs fest. Das neuronale Netz wird
    # soweit aufgefaltet wie wir Wörter als Input Sequenz in das Model rein geben
    for t in range(seq_length):
                
        # Hier wird über den Input und die generierten Targets iteriert
        # vielleicht fällt uns hier noch ne andere art ein das zu machen? 
        input_i = hot_input[t,:]
        input_i = tf.expand_dims(input_i, axis=0)
        
        # wofür brauchen wir den? 
        target_i = hot_target[t,:]
        
        # Compute the new hidden state.
        # Hier wird der neue hidden state für den nächsten Input generiert
        # Als Aktivierungsfunktion benutzen wir die tanh() funktion
        new_hidden = tf.tanh(tf.matmul(input_i, weights_in) + tf.matmul(new_hidden, weights_hidden) + bias_hidden)
        # Compute the new output.
        new_log = tf.matmul(new_hidden, weights_out) + bias_out
        # Store hidden state and output.
        hidden_list.append(new_hidden)
        logits.append(new_log)
        
# In remember merken wir uns den letzten hidden state, damit wir ihn
# als hidden state ins Model geben können, wenn die nächste subsequence rein kommt 
remember = hidden_list[0]

print(remember)

Tensor("simple_RNN_layer/Tanh:0", shape=(1, 100), dtype=float32)


### Der Output, der Loss und der Optimizer

In [104]:
# Wir berechnen den Output, indem wir die softmax Funktion auf die Logits anwenden. 
# Diese befinden sich in als letztes in der Liste aller Logits
out = tf.nn.softmax(logits[-1])

# wir benötigen die Berechnung von cross entropy um den Loss zu berechnen
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
    labels = hot_target,
    logits = logits)
print(cross_entropy)

# Nun nehmen wir den mean von der cross entropy, um einen Wert für den Loss zu erhalten
loss = tf.reduce_mean(cross_entropy)

print(cross_entropy)
print(loss)

#? wofür ist das? 
#outputs = tf.concat(logits, axis=0)
#print(outputs)

Tensor("softmax_cross_entropy_with_logits/Reshape_2:0", shape=(30, 1), dtype=float32)
Tensor("softmax_cross_entropy_with_logits/Reshape_2:0", shape=(30, 1), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


In [105]:
# Mit der Lernrate haben wir ein bisschen rumgespielt. 
# Im Endeffekt haben wir uns hier für entschieden
learning_rate = 1e-4

# Wir haben uns, wie in dem Keras Tutorial für den Adam Optimizer entschieden
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Definiere, was die Optimierungsfunktion minimieren soll. 
# Hier ist es der Loss
training_step = optimizer.minimize(loss)

### Training des Models

In [ ]:
# Lege fest wie viele Epochen das Model trainieren soll
epochs = 200

with tf.Session() as sess:
    
    # Ein globaler Zähler, in welcher Epoche wir uns befinden
    global_step = 0
    sess.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver()
    
    for _ in range(epochs):
    
       
        # Load the dataset into the iterator.
        sess.run(iterator_init_op)
        generated_words = np.zeros([1,nr_hidden_neurons], dtype = np.float32)

        # Wir gehen so lange durch die Schleife, bis unsere Input daten einmal durch gelaufen sind
        while True:
            try:
                # brauchen wir das hier? Irgendwie war es in der letzten Zeit ausdokumentiert.
                #If we are in the first time step intialize the hidden state with zeros.
                #if t == 0:
                #    hs_remember_val = np.zeros([1,hidden_size])
                
                # Feed in the last hidden state.
                # Read out the loss value for printing.
                # Read out the hidden state for the next forward step.
                # Do the training step.
                _, loss_value, generated_words = sess.run([training_step, loss, remember], feed_dict={hidden_state: generated_words})
                
          
                # Zähle den globalen Zähler eins hoch
                global_step += 1

            # Diese Schleife wird beendet, sobald der Iterator leer ist
            except tf.errors.OutOfRangeError:
                break
                
          
        # Hier wird das Model abgespeichert 
        saver.save(sess, "./checkpoint/model.ckpt", global_step = global_step)
        
        
        # Printe nach jeder Epoche die wie vielte Epoche es ist und wie hoch der Training loss ist
        print("Nr. of Epochs: {}, Training loss: {:f}".format(epoch, loss_value))
        
        
        # Hier legen wir fest, wie viele Wörter gesampled werden sollen. 
        # Ein Tweet besteht aus bis zu 140 Zeichen, daher haben wir Wörtern um die 20 gespielt
        sample_len = 30
        
        # Wir beginnen mit einer zufälligen Sequenz aus unsererm Input
        start = random.randint(0, len(text_as_id) - seq_length)
        random_sequence = text_as_id[start:start + seq_length]      
      
        #Hier werden die Wörter gespeichert, die unser Model generiert/sampled
        sampled_words = []
        generated_words = np.zeros([1,nr_hidden_neurons])

        # In dieser for Schleife werden so viele Wörter neu generiert, wie wir durch die 
        # Variable "sample_len" im Voraus festlegen
        for n in range(sample_len):
            
            # To feed the starting sequence into our model we first need to put it into a dataset.
            # As we do not compare anything here we need some fake target values.
            fake_target = np.zeros([1,30], dtype=np.int32)
            sample_dataset = tf.data.Dataset.from_tensor_slices(([random_sequence], fake_target))
           
            # Hier laden wir das Datenset in den Iterator
            sess.run(iterator.make_initializer(sample_dataset))
    
            # Zuerst lesen wir den Output der softmax Funktion aus. Das ist nämlich das nächste Wort, 
            # welches unser Model uns ausgibt. Außerdem müssen wir den letzten hidden state auslesen,
            # damit wir ihn für die nächste sequenz wieder ins Model rein geben können 
            
            sample_output_softmax_val, sample_hs_remember_val = sess.run([out, remember],
                                                                       feed_dict={hidden_state: sample_hs_remember_val})

            
            
            # Sample a character from the softmax output distribution and append it.
            sample = np.random.choice(range(vocab_size), p=sample_output_softmax_val.ravel())
            
            # füge das neue Wort zu den bereits generierten hinzu
            sampled_words.append(sample)
            # Update the start sequence for sampling the next character
            # An dieser Stelle wird die Input- Start - Sequenz erneuert, und um ?? 
            random_sequence = random_sequence[1:] + [sample]
        
      
        # Gib uns einen Tweet aus. 
        new_tweet = ' '.join(id_to_word[i] for i in sampled_words)
        print('----\n %s \n----\n' % (new_tweet,))    
            
        